In [1]:
import pandas as pd
import numpy as np
import ast
import geopy.geocoders
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Vytvoření tabulky s geolokačními údaji (LineString, MultilineString) rozdělenými po jednotlivých bodech zvlášť na řádky

načtení souboru, odmazání sloupce type, přejmenování názvů sloupců

In [2]:
trafficDelaysDf = pd.read_csv('C:/Users/verav/Documents/Projekt/Python_projekt/plynulostVSE.csv')

In [3]:
trafficDelaysDf = trafficDelaysDf.drop(columns = ['type'])
trafficDelaysDf.columns = trafficDelaysDf.columns.str.replace('properties.','')
trafficDelaysDf= trafficDelaysDf.rename(columns={'geometry.type':'coordType'})
trafficDelaysDf= trafficDelaysDf.rename(columns={'geometry.coordinates':'coordinates'})

převod souřadnic ze str na seznam seznamů

In [4]:
coordList = []
for i in trafficDelaysDf.index:
    nestList = ast.literal_eval(trafficDelaysDf["coordinates"][i])
    coordList.append(nestList)
trafficDelaysDf2 = trafficDelaysDf.copy() #tvorba kopie původní tabulky
trafficDelaysDf2['coordinates']=coordList #náhrada původního sloupce coordinates

In [23]:
trafficDelaysDf2.head()

,uuid,pubMillis,level,delay,speed,speedKMH,length,street,endNode,city,country,type,roadType,turnType,blockingAlertUuid,objectid,globalid,coordType,coordinates
0,1363839630,2020/07/13 02:26:04.341+00,5,-1,0,NaN,162,» Centrum / Bohunice,NaN,Brno,EZ,NONE,4,NONE,NaN,1,{DBAD85B2-B703-9D6C-CAD9-160A683F236F},LineString,"[[16.558271, 49.171838], [16.558371, 49.17184]..."
1,1388730276,2020/07/13 02:26:01.618+00,5,-1,0,NaN,45,NaN,NaN,Brno,EZ,NONE,20,NONE,aff5a9d0-5b84-3878-94a5-4c8c39ccfe4c,2,{988271DB-76D7-305B-2CDB-84D662DB0287},LineString,"[[16.567145, 49.171995], [16.567112, 49.171732..."
2,1451737922,2020/07/13 02:26:03.702+00,5,-1,0,NaN,52,U Dráhy,NaN,Ostopovice,EZ,NONE,1,NONE,f5987a0c-c5e0-3064-ad5d-9d58b296b1d9,3,{45B64E51-1EA0-BCB3-BEF6-D79B9FD498FC},LineString,"[[16.544367, 49.16387], [16.544937, 49.163597]]"
3,1388985166,2020/07/13 02:27:45.411+00,5,-1,0,NaN,19,Na Hrázi,NaN,Střelice,EZ,NONE,1,NONE,a7ae280a-44f3-36cc-a6ce-8d5720ac4e37,4,{4BE78E6E-7162-7B43-A291-0A02B59D2BD6},LineString,"[[16.501287, 49.153107], [16.501311, 49.153273]]"
4,1440208034,2020/07/13 02:26:06.591+00,5,-1,0,NaN,44,NaN,NaN,Brno,EZ,NONE,20,NONE,c2b5e9f5-8eef-3e08-9092-ba52f4726a62,5,{25B0DAE6-5F52-657F-50D9-4E20B8F16303},LineString,"[[16.565195, 49.168989], [16.565247, 49.169382]]"


rozšíření tabulky o řádky s jednotlivými souřadnicemi

In [6]:
newList = []
allrows =trafficDelaysDf2.values.tolist()#prevod hodnot v tabulce na seznam allrows
for line in allrows:
    newList += [line[:-1] + [coor] for coor in line[-1]] #kod od Martina, list comprehension misto explode

newListMulti = [] # další zanoření pro multilinestring
for line in newList:
    if 'MultiLineString' in line:
        newListMulti += [line[:-1] + [coor] for coor in line[-1]]
    else:
        newListMulti.append(line)

dataFrame3 ze seznamu

In [ ]:
trafficDelaysDf3= pd.DataFrame(newListMulti)
trafficDelaysDf3.head()
#výpis hodnot sloupce coordinates pro multilinestring
#trafficDelaysDf3[18][trafficDelaysDf3[17]=='MultiLineString'].head()


přiřazení původních názvů sloupců a kontrola

In [26]:
trafficDelaysDf3.columns =trafficDelaysDf.columns

In [27]:
trafficDelaysDf3.shape
trafficDelaysDf3.head()

,uuid,pubMillis,level,delay,speed,speedKMH,length,street,endNode,city,country,type,roadType,turnType,blockingAlertUuid,objectid,globalid,coordType,coordinates
0,1363839630,2020/07/13 02:26:04.341+00,5,-1,0,NaN,162,» Centrum / Bohunice,NaN,Brno,EZ,NONE,4,NONE,NaN,1,{DBAD85B2-B703-9D6C-CAD9-160A683F236F},LineString,"[16.558271, 49.171838]"
1,1363839630,2020/07/13 02:26:04.341+00,5,-1,0,NaN,162,» Centrum / Bohunice,NaN,Brno,EZ,NONE,4,NONE,NaN,1,{DBAD85B2-B703-9D6C-CAD9-160A683F236F},LineString,"[16.558371, 49.17184]"
2,1363839630,2020/07/13 02:26:04.341+00,5,-1,0,NaN,162,» Centrum / Bohunice,NaN,Brno,EZ,NONE,4,NONE,NaN,1,{DBAD85B2-B703-9D6C-CAD9-160A683F236F},LineString,"[16.560043, 49.172452]"
3,1363839630,2020/07/13 02:26:04.341+00,5,-1,0,NaN,162,» Centrum / Bohunice,NaN,Brno,EZ,NONE,4,NONE,NaN,1,{DBAD85B2-B703-9D6C-CAD9-160A683F236F},LineString,"[16.560231, 49.172505]"
4,1388730276,2020/07/13 02:26:01.618+00,5,-1,0,NaN,45,NaN,NaN,Brno,EZ,NONE,20,NONE,aff5a9d0-5b84-3878-94a5-4c8c39ccfe4c,2,{988271DB-76D7-305B-2CDB-84D662DB0287},LineString,"[16.567145, 49.171995]"


kontrolní tabulka před rozšířením o multilinestring

In [10]:
trafficDelaysDf4= pd.DataFrame(newList)
trafficDelaysDf4.shape

(1316407, 19)

uložení tabulky df3 do csv v kompletní a zkrácené verzi

In [ ]:
trafficDelaysDf3.to_csv('trafficDf3.csv')
trafficDelaysDf3.head(1000).to_csv('trafficDf_1000.csv')

## PŘÍPRAVA pro přiřazení názvů ulic podle souřadnic

In [11]:
geopy.geocoders.options.default_user_agent = 'Mozilla/5.0'
geolocator = Nominatim()

In [12]:
type(trafficDelaysDf3['coordinates'][0][0])

float

načtení zestručněné tabulky trafficDelaysDf (pouze uuid a coordinates)

In [17]:
trafficDelaysCoord = pd.read_csv("coord_uuid.csv")

nahrazení NaN ve sloupci street v původní tabulce "ničím"

In [14]:
trafficDelaysDf["street"].fillna('', inplace=True)

In [15]:
trafficDelaysDf["street"].head()

0    » Centrum / Bohunice
1                        
2                 U Dráhy
3                Na Hrázi
4                        
Name: street, dtype: object

streetNaN - výpis sloupce uuid pro podmínku, kde street je prázdná

In [18]:
streetNaN = trafficDelaysDf["uuid"][trafficDelaysDf["street"]==""]

In [ ]:
trafficDelaysCoord["uuid"].isin(streetNaN)


tabulka prázdných ulic

In [20]:
trafficDelaysCoordNaN = trafficDelaysCoord[trafficDelaysCoord["uuid"].isin(streetNaN)]
trafficDelaysCoordNaN.head()
#trafficDelaysCoordNaN.to_csv('missing.csv')

,uuid,coordinates
1,1388730276,"[16.567145, 49.171995]"
4,1440208034,"[16.565195, 49.168989]"
5,1370182069,"[16.567319, 49.17172]"
6,1415306095,"[16.565247, 49.169382]"
8,1415188035,"[16.565195, 49.168989]"


script pro přiřazení názvů - 1.kolo (podle první souřadnice v seznamu)

In [ ]:
streetList = []
for rows in trafficDelaysCoordNaN['coordinates']:
    souradnice = str(row[1])+','+ str(row[0])  
    location = geolocator.geocode(souradnice, language = "cz")
    nazev = location.raw['display_name']
    ulice = nazev.split(",")
    if len(ulice) == 9:
        streetList.append(ulice[1])
    elif len(ulice) == 8:
        streetList.append(ulice[0])
    else:
        streetList.append(ulice)
    
    #druhý způsob geocoding, řešeno v streetNames.ipynb:   
    #geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    #df['location'] = df['name'].apply(geocode)